In [134]:
import pandas as pd
import nltk
import numpy as np
import re

## Carregando os dados

In [135]:
birdie_data = pd.read_csv('data/resultados.tsv', sep='\t')
submarino_data = pd.read_csv('data/submarino_smartphones.csv')

In [136]:
submarino_data.head()

,url,smartphone
0,https://www.submarino.com.br/categoria/celular...,Smartphone Motorola One Vision 128GB Dual Chip...
1,https://www.submarino.com.br/categoria/celular...,"Iphone 8 Plus 64GB Ouro Tela 5.5"" iOS 12 4G C..."
2,https://www.submarino.com.br/categoria/celular...,iPhone 8 Plus 64gb Silver Tela 5.5” iOS 12 4G ...
3,https://www.submarino.com.br/categoria/celular...,Smartphone Samsung Galaxy S10e 128GB Dual Chip...
4,https://www.submarino.com.br/categoria/celular...,Smartphone Samsung Galaxy Note 9 128GB Nano Ch...


In [137]:
birdie_data.head()

,ID,TITLE,SMARTPHONE
0,1041354,Acessório T - Jean Bag For Girls para DS Lite,0.0
1,1041782,Carrinho de Bebê Berço-Passeio - Pegasus Pink ...,0.0
2,1041834,Carrinho de Bebê para Gêmeos Berço-Passeio - T...,0.0
3,1042568,Car Center - Calesita,0.0
4,1042584,Donka Trem com Som - Calesita,0.0


In [138]:
print('tamanho:', len(submarino_data['smartphone']))
print('qtd de smartphones unicos:', submarino_data['smartphone'].nunique())

tamanho: 4680
qtd de smartphones unicos: 3803


In [139]:
birdie_data = pd.DataFrame(birdie_data['TITLE'][birdie_data['SMARTPHONE'] == 1], columns=['TITLE'])

In [140]:
print('tamanho:', len(birdie_data['TITLE']))
print('qtd de smartphones unicos:', birdie_data['TITLE'].nunique())

tamanho: 588
qtd de smartphones unicos: 546


In [141]:
submarino_data = pd.DataFrame(submarino_data['smartphone'])
submarino_data.columns = ['TITLE']

In [142]:
submarino_data['TITLE'] = submarino_data['TITLE'].apply(lambda x: x.lower())

In [143]:
submarino_data.head()

,TITLE
0,smartphone motorola one vision 128gb dual chip...
1,"iphone 8 plus 64gb ouro tela 5.5"" ios 12 4g c..."
2,iphone 8 plus 64gb silver tela 5.5” ios 12 4g ...
3,smartphone samsung galaxy s10e 128gb dual chip...
4,smartphone samsung galaxy note 9 128gb nano ch...


In [144]:
birdie_data['TITLE'] = birdie_data['TITLE'].apply(lambda x: x.lower())

In [145]:
birdie_data.head()

,TITLE
6,smartphone lg optimus l7 ii dual p716 preto co...
7,smartphone samsung galaxy s4 mini duos preto c...
8,smartphone samsung galaxy s5 sm-g900m branco ...
9,smartphone samsung galaxy s5 sm-g900m dourado...
10,smartphone lg l90 dual d410 preto com tela de ...


In [395]:
re_marcas = ('samsung|lg|motorola|asus|apple|multilaser|sony|xiaomi|' + 
             'alcatel|positivo|microsoft|lenovo|huawei|caterpillar')

re_qtds_memoria = '\d{1}gb'

re_qtds_armazenamento = '\d{2,3}gb'

re_cor = 'azul|amarelo|vermelho|branco|preto|dourado|prata|violeta|grafite|black piano'

# re_dual_chip = 'dual chip|dual sim'

re_tam_tela = '[4-7][\.,]\d'

re_modelos = ('galaxy \w\d{1,2} prime|mini|galaxy \w\d{1,2}\w?\+?|galaxy .* duos|galaxy note \d|' + # Samsung
              'moto \w\d? plus|moto \w\d?|' + # Motorola
              'redmi note \d\w? lite|redmi note \d\w?|mi \w?\d|mi \w?\d lite|pocophone \w\d{1,2}|' + # Xiaomi
              'iphone [\dx] plus|iphone [\dx]|' + # Apple
              'lg k\d{1,3}|' + # LG
              'zenfone \d|' + # Asus
              'cat \w\d{1,2}|' + # Caterpillar
              'xperia \w{1,2}\d plus \w*\d{1,4}|xperia \w{1,2}\d \w*\d{1,4}|' + # Sony
              'vibe k\d \w*|k\d \w*')

res = {'MARCA': re_marcas,
       'MODELO': re_modelos,
       'QTD_MEMORIA': re_qtds_memoria,
       'QTD_ARMAZENAMENTO': re_qtds_armazenamento,
       }

# Removendo as cores
smartphones = []
for smartphone in birdie_data['TITLE']:
    encontrado = re.findall(re_cor, smartphone)
    if encontrado:
        encontrado = encontrado.pop()
        smartphones.append(smartphone.replace(encontrado+' ', ''))
    else:
        smartphones.append(smartphone)
        
birdie_data['TITLE'] = smartphones

In [396]:
for atributo, regex in res.items():
    i = 0
    encontrados = []
    for smartphone in birdie_data['TITLE']:
        encontrado = re.findall(regex, smartphone)
        if encontrado:
            encontrados.append(encontrado[0])
        else: 
            encontrados.append('')
        i += 1
    
    birdie_data[atributo] = encontrados
    birdie_data.reset_index(drop=True,inplace=True)

In [397]:
birdie_data['MARCA'].value_counts()

samsung        150
motorola       125
xiaomi          96
apple           67
                57
lg              32
asus            27
caterpillar      9
sony             8
lenovo           5
alcatel          4
multilaser       3
huawei           2
positivo         2
microsoft        1
Name: MARCA, dtype: int64

In [398]:
birdie_data['MODELO'].value_counts().sum()

588

In [399]:
birdie_data.head()

,TITLE,MARCA,MODELO,QTD_MEMORIA,QTD_ARMAZENAMENTO
0,smartphone lg optimus l7 ii dual p716 com dual...,lg,,4gb,
1,smartphone samsung galaxy s4 mini duos com dua...,samsung,galaxy s4 m,,
2,smartphone samsung galaxy s5 sm-g900m com tel...,samsung,galaxy s5,,
3,smartphone samsung galaxy s5 sm-g900m com tel...,samsung,galaxy s5,,
4,"smartphone lg l90 dual d410 com tela de 4.7”, ...",lg,,,


In [400]:
for atributo, regex in res.items():
    i = 0
    encontrados = []
    for smartphone in submarino_data['TITLE']:
        encontrado = re.findall(regex, smartphone)
        if encontrado:
            encontrados.append(encontrado[0])
        else: 
            encontrados.append('')
        i += 1
    
    submarino_data[atributo] = encontrados
    submarino_data.reset_index(drop=True,inplace=True)

In [401]:
birdie_smartphones = []
smartphone = ''
for i in range(len(birdie_data)):
    for attribute in birdie_data.columns[1:]:
        smartphone += str(birdie_data[attribute][i]) + ' '
        
    smartphone = smartphone.strip()
    birdie_smartphones.append(smartphone)
    smartphone = ''

In [402]:
birdie_data

,TITLE,MARCA,MODELO,QTD_MEMORIA,QTD_ARMAZENAMENTO
0,smartphone lg optimus l7 ii dual p716 com dual...,lg,,4gb,
1,smartphone samsung galaxy s4 mini duos com dua...,samsung,galaxy s4 m,,
2,smartphone samsung galaxy s5 sm-g900m com tel...,samsung,galaxy s5,,
3,smartphone samsung galaxy s5 sm-g900m com tel...,samsung,galaxy s5,,
4,"smartphone lg l90 dual d410 com tela de 4.7”, ...",lg,,,
5,smartphone samsung galaxy core plus com tela ...,samsung,,,
6,smartphone samsung galaxy pocket 2 duos sm-g11...,samsung,galaxy pocket 2 duos,,
7,smartphone samsung galaxy s5 duos sm-g900 com...,samsung,galaxy s5,,
8,smartphone samsung galaxy ace 4 lite duos g313...,samsung,galaxy ace 4 lite duos,,
9,"smartphone lg g2 lite d295 com tela de 4.5”, d...",lg,,,


In [303]:
submarino_smartphones = []
smartphone = ''
for i in range(len(submarino_data)):
    for attribute in submarino_data.columns[1:]:
        smartphone += str(submarino_data[attribute][i]) + ' '
        
    smartphone = smartphone.strip()
    submarino_smartphones.append(smartphone)
    smartphone = ''

In [260]:
matches = []
i = 0
for smartphone in birdie_smartphones:
    matches.append(process.extractOne(smartphone, submarino_smartphones))
    print(i)
    i += 1
    if i == 20:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [265]:
matches

[('lg  4gb', 100),
 ('samsung galaxy s4', 100),
 ('samsung galaxy s5', 100),
 ('samsung galaxy s5', 100),
 ('lg', 100),
 ('samsung', 100),
 ('samsung galaxy pocket duos', 96),
 ('samsung galaxy s5', 100),
 ('samsung', 90),
 ('lg', 100),
 ('samsung galaxy young 2 duos', 100),
 ('samsung galaxy young 2 duos', 100),
 ('alcatel', 100),
 ('samsung galaxy win 2 duos', 100),
 ('samsung galaxy ace 4 neo duos 4gb', 95),
 ('samsung galaxy a7', 95),
 ('samsung galaxy ace 4 neo duos 4gb', 95),
 ('samsung galaxy e5 4g duos', 100),
 ('samsung galaxy e5 4g duos', 96),
 ('samsung galaxy s6 4gb 64gb', 100)]

In [267]:
indices = []
for match in matches:
    indices.append(submarino_smartphones.index(match[0]))

In [274]:
df = pd.DataFrame(columns=['birdie','submarino'])
df['birdie'] = birdie_data.iloc[:20, 0]
df['submarino'] = submarino_data.iloc[indices, 0].reset_index(drop=True)

In [275]:
df

,birdie,submarino
0,smartphone lg optimus l7 ii dual p716 com dual...,smartphone lg joy claro dual chip desbloqueado...
1,smartphone samsung galaxy s4 mini duos com dua...,capa protetora samsung flip cover para galaxy s4
2,smartphone samsung galaxy s5 sm-g900m com tel...,usado: samsung galaxy s5 mini duos preto excel...
3,smartphone samsung galaxy s5 sm-g900m com tel...,usado: samsung galaxy s5 mini duos preto excel...
4,"smartphone lg l90 dual d410 com tela de 4.7”, ...",smartphone lg h422
5,smartphone samsung galaxy core plus com tela ...,smartwatch samsung galaxy watch active sm-r500...
6,smartphone samsung galaxy pocket 2 duos sm-g11...,smartphone samsung galaxy pocket duos s5302 br...
7,smartphone samsung galaxy s5 duos sm-g900 com...,usado: samsung galaxy s5 mini duos preto excel...
8,smartphone samsung galaxy ace 4 lite duos g313...,smartwatch samsung galaxy watch active sm-r500...
9,"smartphone lg g2 lite d295 com tela de 4.5”, d...",smartphone lg h422
